In [1]:
import pyvista as pv
import pyvista.examples
import torch
opts = {'jupyter_backend': 'pythreejs', 'window_size' : [800, 350]}
import numpy as np
import math
import tetgen

Need for Tetgen

In [2]:
!pip install pymeshfix #### Need for Tetgen

In [3]:
import pymeshfix

## Defining function and constants##

\begin{array}{lccc}
\hline \text { Parameter } & \text { Symbol } & \text { Value } & \text { Units } \\
\hline \text { Brine density } & \rho_{\mathrm{w}} & 1100 & \mathrm{~kg} \mathrm{~m}^{-3} \\
\text { Brine viscosity } & \mu_{\mathrm{w}} & 900 & \mu \mathrm{Pa} \mathrm{s} \\
\mathrm{CO}_{2} \text { density } & \rho_{\mathrm{n}} & 720 & \mathrm{~kg} \mathrm{~m}^{-3} \\
\mathrm{CO}_{2} \text { viscosity } & \mu_{\mathrm{n}} & 60 & \mu \mathrm{Pa} \mathrm{s} \\
\text { Rock density } & \rho_{\mathrm{s}} & 2670 & \mathrm{~kg} \mathrm{~m}^{-3} \\
\text { Rock porosity } & \phi & 0.15 & - \\
\mathrm{CO}_{2} \text { relative permeability } & k_{\mathrm{rn}} & 0.3948 & - \\
\text { Brine relative permeability } & k_{\mathrm{rw}} & 1 & - \\
\text { Brine residual saturation } & S_{\mathrm{rw}} & 0.4438 & - \\
\text { Well/reservoir height } & H & 170 & \mathrm{~m} \\
\mathrm{CO}_{2} \text { mass injection rate } & M_{\mathrm{n}} & 20 & \mathrm{~kg} \mathrm{~s}^{-1} \\
\hline
\end{array}

In [6]:
k_rn = 0.3948
k_rw = 1
S_rw= 0.4438
mu_n = 60
mu_w = 900
rho_n = 720
rho_s = 2670
phi = 0.15
H = 1 ###!!!!! Indeed it is not real H. I used it because all warks with it !!!!!
M_n = 20
gamma = ((mu_n)*(k_rw))/((mu_w)*(k_rn))
theta = ((M_n)/((math.pi)*H*phi*(1-S_rw)*(rho_n)*gamma))**(0.5)
t=1
t2 = 10

$$ h(x, t) \approx \begin{cases}H & ; x \leq \gamma \theta \sqrt{t} \\ \frac{H \gamma \theta}{1-\gamma} \frac{\sqrt{t}}{x}-\frac{H \gamma}{1-\gamma} & ; \gamma \theta \sqrt{t}<x<\theta \sqrt{t} \\ 0 & ; \theta \sqrt{t} \leq x\end{cases} $$

$$
\gamma=\frac{\mu_{\mathrm{n}} k_{\mathrm{rw}}}{k_{\mathrm{rn}} \mu_{\mathrm{w}}} , \theta=\sqrt{\frac{M_{n}}{\pi H \phi\left(1-S_{\mathrm{rw}}\right) \rho_{\mathrm{n}} \gamma}}
$$

In [7]:
def plum_thikness2d (H, gamma, theta, m , t):
    if (m <= (gamma*theta*t**(0.5))):
        h = H
    if ((m > (gamma*theta*(t**(0.5)))) and (m < (theta*(t**(0.5)))) ):
        h = ((H*gamma*theta)/(1-gamma))*(t**(0.5))/(m) - (H*gamma)/(1-gamma)
    if (m >= (theta*t**(0.5))) :
        h = 0
    return h

Create array for our manifold. xyz - for layer. It is better to run only one layer first time

In [8]:
xyz = np.zeros((20,20,20)) #создаем CO2 слой в момент времени 1

#xyz2 = np.zeros((20,20,20))#создаем CO2 слой в момент времени 2

Fill the array 


In [15]:
func = [] 
for i in range(20):
    for j in range(20):
        var_y = -1 + j*0.1
        for k in range(20):
            var_x = -1 + k*0.1
            var = var_x * var_x + var_y * var_y # радиус x^2+y^2
            mass = np.asarray([var_x,var_y,1 - plum_thikness2d (H, gamma, theta, var , t)]) # see  description **
            func.append(mass) ##   see  description**
            xyz[i][j][k] = plum_thikness2d (H, gamma, theta, var , t) # This type of array we need if we wont color mesh ( this
            # way like in you notebook (pyvista)

** We need this because as a result we want to have array of point wich looks like: [ (x1,y1,z1), (x2,y2,z2) ... (xn,yn,zn) ]. Because it is type of PolyData that we need. After we convert list to array

Use it if you want add another mesh

In [70]:
"""
func2 = [] 
for i in range(20):
    for j in range(20):
        var_y2 = -1 + j*0.1
        for k in range(20):
            var_x2 = -1 + k*0.1
            var2 = var_x2 * var_x2 + var_y2 * var_y2 # радиус x^2+y^2
            mass2 = np.asarray([var_x2,var_y2,plum_thikness2d (H, gamma, theta, var2 , t2)])
            xyz2[i][j][k] = plum_thikness2d (H, gamma, theta, var2 , t2)
            func2.append(mass2)
"""            

'\nfunc2 = [] \nfor i in range(20):\n    for j in range(20):\n        var_y2 = -1 + j*0.1\n        for k in range(20):\n            var_x2 = -1 + k*0.1\n            var2 = var_x2 * var_x2 + var_y2 * var_y2 # радиус x^2+y^2\n            mass2 = np.asarray([var_x2,var_y2,plum_thikness2d (H, gamma, theta, var2 , t2)])\n            xyz2[i][j][k] = plum_thikness2d (H, gamma, theta, var2 , t2)\n            func2.append(mass2)\n'

In [16]:
func = np.asarray(func) ##We had list but we need array
#func2 = np.asarray(func2) ## Use it if you want add another mesh

In [17]:
point_cloud = pv.PolyData(func)
#point_cloud2 = pv.PolyData(func2) # Use it if you want add another mesh

In [18]:
#point_cloud2.plot(point_size=10, style='points', **opts) ## Use it if you want add another mesh

At firs we need to create surface of C02. Now it is just 2d triangulation. We will use it to create 3d 

In [19]:
tmesh = point_cloud.delaunay_2d()
#tmesh2 = point_cloud2.delaunay_2d() ## Use it if you want add another mesh

To be sure about we have normal manifold 

In [22]:
pl = pv.Plotter()
pl.add_mesh(tmesh, show_edges=True, color='white', opacity=0.5)
pl.add_points(tmesh.points, color='red', point_size=20)
pl.show(**opts)

Renderer(camera=PerspectiveCamera(aspect=2.2857142857142856, children=(DirectionalLight(color='#fefefe', inten…

To make 3d triangulation we need to close this surface. This could be done by pymesh fix. As a result our surface will become manifold. This is important beacuse tetgen works only with manifold

In [24]:
fixer = pymeshfix.MeshFix(tmesh)
fixer.repair(joincomp=True, remove_smallest_components=False)
#fixer2 = pymeshfix.MeshFix(tmesh2) ## Use it if you want add another mesh
#fixer2.repair(joincomp=True, remove_smallest_components=False) ## Use it if you want add another mesh

In [26]:
fixer.mesh.plot(**opts)

Renderer(camera=PerspectiveCamera(aspect=2.2857142857142856, children=(DirectionalLight(color='#fefefe', inten…

We created proper manifold 

In [27]:
newmesh = fixer.mesh
#newmesh2 = fixer2.mesh ## Use it if you want add another mesh

## 3d triangulation with Tetgen ##

It lookds like 2d but indeed is 3d. You can exptract cell and see that there are sell in this manifold

In [40]:
tett  = tetgen.TetGen(newmesh )
tett.tetrahedralize(order=1, mindihedral=20, minratio=1.5)
grid = tett.grid
#tett2  = tetgen.TetGen(newmesh2 ) ##Use it if you want add another mesh
#tett2.tetrahedralize(order=1, mindihedral=20, minratio=1.5) ##
#grid2 = tett2.grid ##
grid.plot(show_edges=True, opacity = 0.8, **opts)

Renderer(camera=PerspectiveCamera(aspect=2.2857142857142856, children=(DirectionalLight(color='#fefefe', inten…

You need to scroll away a little to see picture and rotate. After you will slice with see triangulation

In [41]:
#1. get cell centroids. Th
cells = grid.cells.reshape(-1, 5)[:, 1:]
cell_center = grid.points[cells].mean(1)

#2. extract cells below the 0 xy plane
mask = cell_center[:, 1] < 0.1
cell_ind = mask.nonzero()[0]
subgrid = grid.extract_cells(cell_ind)

# get cell centroids
#cells2 = grid2.cells.reshape(-1, 5)[:, 1:]
#cell_center2 = grid2.points[cells2].mean(1)

# extract cells below the 0 xy plane
#mask2 = cell_center2[:, 1] < 0.1
#cell_ind = mask2.nonzero()[0]
#subgrid2 = grid2.extract_cells(cell_ind)


#adding soil
soil1 = pyvista.Cube(x_length=4.0, y_length=2.0, z_length=2.0, clean= True)
soil2 = pyvista.Cube(center=(0.0,0.0,-2), x_length=4.0, y_length=2.0, z_length=2.0, bounds=None, clean=True)
soil3 = pyvista.Cube(center=(0.0,0.0,+2), x_length=4.0, y_length=2.0, z_length=2.0, bounds=None, clean=True)

## adding detecotr
detector = pyvista.Cube(center=(0.0,0.0,-3), x_length=0.2, y_length=0.2, z_length=0.2, bounds=None, clean=True)

# postion of camera
plotter = pv.Plotter()
plotter.camera_position ='xy' #(-1, 1.0, 1.0)
#plotter.camera.roll += 10

#CO2 in t = 1

plotter.add_mesh(subgrid, 'grey', lighting=True, show_edges=True)
plotter.add_mesh(grid, 'r', 'wireframe')

#CO2 in t = 10
#plotter.add_mesh(subgrid2, 'black', opacity = 0.7 ,lighting=True, show_edges=True)
#plotter.add_mesh(grid2, 'black', 'wireframe')
#plot soil
plotter.add_mesh(soil1, 'brown', opacity = 0.7)
plotter.add_mesh(soil2, 'gold', opacity = 0.7)
plotter.add_mesh(soil3, 'green', opacity = 0.7)

#plot detecotr
plotter.add_mesh(detector, 'purple')

#plot ray trace
# Define line segment
start = [0, 0, -3]
stop = [1, 1, 4]

# Perform ray trace
a = [0, 0, -3]
b = [1, 1, 4]
line = pv.Line(a, b)
p = pv.Plotter()
plotter.add_mesh(line, color="purple")
#intersection = pv.PolyData(points)
plotter.add_legend([[' Input Mesh ', 'r'],
                    [' Tessellated Mesh ', 'black']])
plotter.show(**opts)

Renderer(camera=PerspectiveCamera(aspect=2.2857142857142856, children=(DirectionalLight(color='#fefefe', inten…

## Way you suggest ##

 I tried but i have sufficient progress

In [32]:
grid = pv.UniformGrid()
# Set the spatial reference
grid.dimensions = np.array(xyz.shape) +1 #dim + 1 because cells
grid.origin = (0, 0, 0)
grid.spacing = (1, 1, 1)

In [33]:
grid['data'] = xyz.ravel(order='F')

In [34]:
grid.plot(show_grid=True, show_edges=True, opacity=0.4, **opts)

Renderer(camera=PerspectiveCamera(aspect=2.2857142857142856, children=(DirectionalLight(color='#fefefe', inten…

In [35]:
gridpoints = pv.PolyData(grid.points)
gridpoints.plot(point_size=20, style='points', **opts)

Renderer(camera=PerspectiveCamera(aspect=2.2857142857142856, children=(DirectionalLight(color='#fefefe', inten…

In [36]:
tmesh = gridpoints.delaunay_3d()

In [57]:
pl = pv.Plotter()
pl.add_mesh(tmesh, show_edges=True, color='white', opacity=0.5)
pl.add_points(tmesh.points, color='red', point_size=20)
pl.show(**opts)

Renderer(camera=PerspectiveCamera(aspect=2.2857142857142856, children=(DirectionalLight(color='#fefefe', inten…